In [ ]:
%aiida
from aiida.orm import load_node

In [ ]:

from aiida.orm.querybuilder import QueryBuilder

from aiida.work.process import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData

from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import time
import ase.io
import ase.units as aseu
from ase.data.colors import jmol_colors
import urlparse
import numpy as np
import copy

import re
from collections import OrderedDict

import matplotlib.pyplot as plt
from pprint import pprint

from tempfile import NamedTemporaryFile
from base64 import b64encode

import zipfile
import io
import StringIO


In [ ]:
a = load_node(170364)

In [ ]:
afm_calc = a.get_outputs()[-2]

In [ ]:
a.get_inputs_dict()['structure'].get_ase()

In [ ]:
def load_afm_pp_data(afm_pp_calc):
    df_path = afm_pp_calc.out.retrieved.get_abs_path('df.npy')
    df_vec_path = afm_pp_calc.out.retrieved.get_abs_path('df_vec.npy')
    
    df_data = np.load(df_path)
    df_vec_data = np.load(df_vec_path)
    
    x_arr = df_vec_data[0, 0] + np.linspace(0.0, df_vec_data[1, 0], df_data.shape[2])
    y_arr = df_vec_data[0, 1] + np.linspace(0.0, df_vec_data[2, 1], df_data.shape[1])

    #x_grid, y_grid = np.meshgrid(x_arr, y_arr)
    
    return x_arr, y_arr, df_data

In [ ]:
def load_all(workcalc):
    
    ase_atoms = workcalc.get_inputs_dict()['structure'].get_ase()
    
    if 'Au' not in list(ase_atoms.symbols):
        print("No Au found, skipping")
        return None
    
    afm_pp_calc = workcalc.get_outputs()[-2]
    afm_2pp_calc = workcalc.get_outputs()[-1]
    
    scan_start_z = workcalc.inp.afm_pp_params.dict.scanMin[2]
    ampl = workcalc.inp.afm_pp_params.dict.Amplitude
    dz = workcalc.inp.afm_pp_params.dict.scanStep[2]
    h0 = scan_start_z + ampl/2.0
    
    x_arr, y_arr, df_data = load_afm_pp_data(afm_pp_calc)
    # df_data[h][x][y]
    
    extent = [np.min(x_arr), np.max(x_arr), np.min(y_arr), np.max(y_arr)]
    
    h_arr = h0 + np.arange(df_data.shape[0])*dz
    
    # filter heights
    h_min = 27.80
    h_max = 28.90
    
    filt = np.logical_and(h_arr>=h_min, h_arr<=h_max)
    
    df_data = df_data[filt, :, :]
    h_arr = h_arr[filt]
    
        
    return df_data, h_arr, extent, ase_atoms


In [ ]:
df_data, h_arr, extent, ase_atoms = load_all(load_node(121641))

In [ ]:
h_arr

In [ ]:
qb = QueryBuilder()
qb.append(WorkCalculation, tag='wc', filters={
    'attributes._process_label': 'AfmWorkChain',
    'and':[
        {'attributes': {'has_key': '_sealed'}},
        {'attributes._sealed': {'==': True}}
    ],
    'and':[
        {'extras': {'has_key': 'preprocess_version'}},
        {'extras.preprocess_version': {'==': 1.02}},
        {'extras.preprocess_successful': {'==': True}},
    ]
})
qb.order_by({'wc': {'ctime': 'asc'}})
afms_wcs = qb.all()
len(afms_wcs)

In [ ]:
zip_buffer = io.BytesIO()

with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED, False) as zip_file:
    
    for x in afms_wcs:
        afms_wc = x[0]
        print(afms_wc.pk)
        
        out = load_all(afms_wc)
        if out is None:
            continue
        
        df_data, h_arr, extent, ase_atoms = out
        
        txtdata = StringIO.StringIO()
        ase_atoms.write(txtdata, format='xyz')
        zip_file.writestr("%d.xyz" % afms_wc.pk, txtdata.getvalue())

        npzdata = StringIO.StringIO()

        np.savez_compressed(npzdata, df_data=df_data, h_arr=h_arr, extent=extent)

        zip_file.writestr("%d.npz" % afms_wc.pk, npzdata.getvalue())
    
    
    
with open('test.zip', 'wb') as f:
        f.write(zip_buffer.getvalue())

# tests

In [ ]:
wc = load_node(121641)

In [ ]:
afm_calc = wc.get_outputs()[-2]

In [ ]:
afm_calc.out.retrieved.get_folder_list()

In [ ]:
wc.get_extras()